## Deploy Model as Azure Machine Learning Web Service using MLflow

This example shows you how to use mlflow together with Azure Machine Learning services for deploying a model as a web service. You'll learn how to:

 1. Retrieve a previously trained scikit-learn model
 2. Create a Docker image from the model
 3. Deploy the model as a web service on Azure Container Instance
 4. Make a scoring request against the web service.



## 0. Prerequisites and Set-up

This notebook requires you to first complete the [Use MLflow with Azure Machine Learning for Local Training Run](../train-local/train-local.ipnyb) or [Use MLflow with Azure Machine Learning for Remote Training Run](../train-remote/train-remote.ipnyb) notebook, so as to have an experiment run with uploaded model in your Azure Machine Learning Workspace.

Also install following packages if you haven't already

```
pip install azureml-mlflow pandas
```


In [1]:
#pip install azureml-mlflow

In [2]:
import time
datedujour = time.strftime("%Y-%m-%d")
print(datedujour)

2019-12-09


In [3]:
import mlflow
import azureml.mlflow
import azureml.core
from azureml.core import Workspace

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.76


## 1. Connect to workspace and set MLflow tracking URI

Setting the tracking URI is required for retrieving the model and creating an image using the MLflow APIs.

In [4]:
ws = Workspace.from_config()

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

## 2. Retrieve model from previous run

Let's retrieve the experiment from training notebook, and list the runs within that experiment.

In [5]:
experiment_name = "MLFlow"
exp = ws.experiments[experiment_name]

runs = list(exp.get_runs())
runs

[Run(Experiment: MLFlow,
 Id: MLFlow_1575898279_b8568104,
 Type: azureml.scriptrun,
 Status: Completed), Run(Experiment: MLFlow,
 Id: 8f636596-574c-4ace-93e2-ae12a6ec4700,
 Type: None,
 Status: Completed), Run(Experiment: MLFlow,
 Id: c7295939-551a-4756-8acc-9dbf650474d9,
 Type: None,
 Status: Completed)]

Then, let's select the most recent training run and find its ID. You also need to specify the path in run history where the model was saved. 

In [6]:
runid = runs[0].id
model_save_path = "model"

## 3. Create Docker image

To create a Docker image with Azure Machine Learning for Model Management, use ```mlflow.azureml.build_image``` method. Specify the model path, your workspace, run ID and other parameters.

**MLflow automatically recognizes the model framework as scikit-learn**, and creates the scoring logic and includes library dependencies for you.

Note that the image creation can take several minutes.

In [7]:
%%time
import mlflow.azureml

azure_image, azure_model = mlflow.azureml.build_image(model_uri="runs:/{}/{}".format(runid, model_save_path),
                                                      workspace=ws,
                                                      model_name='diabetes-sklearn-model',
                                                      image_name='diabetes-sklearn-image',
                                                      synchronous=True)

Registering model diabetes-sklearn-model


2019/12/09 14:40:29 INFO mlflow.azureml: Registered an Azure Model with name: `diabetes-sklearn-model` and version: `5`


Creating image


2019/12/09 14:40:31 INFO mlflow.azureml: Building an Azure Container Image with name: `diabetes-sklearn-image` and version: `5`


Running................................................................................................
Succeeded
Image creation operation finished for image diabetes-sklearn-image:5, operation "Succeeded"
CPU times: user 842 ms, sys: 63 ms, total: 905 ms
Wall time: 8min 20s


## 4. Deploy web service

Let's use Azure Machine Learning SDK to deploy the image as a web service. 

First, specify the deployment configuration. Azure Container Instance is a suitable choice for a quick dev-test deployment, while Azure Kubernetes Service is suitable for scalable production deployments.

Then, deploy the image using Azure Machine Learning SDK's ```deploy_from_image``` method.

Note that the deployment can take several minutes.

In [8]:
%%time
from azureml.core.webservice import AciWebservice, Webservice


aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=1, 
                                                tags={"method" : "sklearn"}, 
                                                description='Diabetes model',
                                                location='eastus2')



CPU times: user 0 ns, sys: 62 µs, total: 62 µs
Wall time: 66.3 µs


In [9]:
%%time

# Deploy the image to Azure Container Instances (ACI) for real-time serving
webservice = Webservice.deploy_from_image(
    image=azure_image, workspace=ws, name="diabetes-aci-model", deployment_config=aci_config)


webservice.wait_for_deployment(show_output=True)

Running...................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 235 ms, sys: 47.1 ms, total: 282 ms
Wall time: 4min 26s


> https://ml.azure.com

## 5. Make a scoring request

Let's take the first few rows of test data and score them using the web service

### 5.1 Scoring Endpoint

In [10]:
webservice.scoring_uri

'http://94d70d34-9e0c-4aeb-bf20-53e8fb5d54df.eastus2.azurecontainer.io/score'

### 5.2 Input

MLflow-based web service for scikit-learn model requires the data to be converted to Pandas DataFrame, and then serialized as JSON. 

In [27]:
tests = [
    [0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,
     -0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337],
    [-0.01277963, -0.04464164,  0.06061839,  0.05285819,  0.04796534,
     0.02937467, -0.01762938,  0.03430886,  0.0702113 ,  0.00720652],
    [ 0.03807591,  0.05068012,  0.00888341,  0.04252958, -0.04284755,
     -0.02104223, -0.03971921, -0.00259226, -0.01811827,  0.00720652]]

In [28]:
import json
import pandas as pd

test_rows_as_json = pd.DataFrame(tests).to_json(orient="split")

In [30]:
test_rows_as_json

'{"columns":[0,1,2,3,4,5,6,7,8,9],"index":[0,1,2],"data":[[0.01991321,0.05068012,0.10480869,0.07007254,-0.03596778,-0.0266789,-0.02499266,-0.00259226,0.00371174,0.04034337],[-0.01277963,-0.04464164,0.06061839,0.05285819,0.04796534,0.02937467,-0.01762938,0.03430886,0.0702113,0.00720652],[0.03807591,0.05068012,0.00888341,0.04252958,-0.04284755,-0.02104223,-0.03971921,-0.00259226,-0.01811827,0.00720652]]}'

### 5.3 Output

In [31]:
%%time
predictions = webservice.run(test_rows_as_json)
print("Model Prediction = ", predictions)

Model Prediction =  [235.11371087161342, 246.8015254559213, 163.6854179229191]
CPU times: user 792 µs, sys: 3.43 ms, total: 4.22 ms
Wall time: 288 ms


You can diagnose the web service using ```get_logs``` method.

In [32]:
webservice.get_logs()

'2019-12-09T14:54:00,370466272+00:00 - rsyslog/run \n2019-12-09T14:54:00,373824678+00:00 - gunicorn/run \n2019-12-09T14:54:00,375149781+00:00 - iot-server/run \n2019-12-09T14:54:00,375948882+00:00 - nginx/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2019-12-09T14:54:00,630162229+00:00 - iot-server/finish 1 0\n2019-12-09T14:54:00,635952739+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.6.0\nListening at: http://127.0.0.1:31311 (10)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 45\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user\'s init function\n2019-12-09 14:54:06,912 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:\n\tMessage: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.\n\tInnerExcepti